In [1]:
import mypaengine as pa
import myqengine as qe
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

pa_doc = pa.DocumentTemplate(pa_document_name='PERSONAL:API_REPORTS_SINGLE',
                            pa_component_category='main / Contribution No Group',
                            pa_component_name='Contribution',
)

In [2]:


dfd = pd.read_csv('nvidia_holder_analysis_full_dataset.csv')
dfd = dfd.sort_values('aum',ascending= False)
#Get pa compatible tickers
fund_univ = qe.id_universe(ids = dfd['ticker'].to_list(),universe_type ='Equity')
time_series = qe.time_series(start_date='0M',end_date = '0M')

formulas = ['LSS_TICKER','FFD_BMK_ID']

q_req = qe.calculate(universe=fund_univ, dates = time_series, formulas=formulas,source= 'ScreeningExpression')
dff = q_req.data.copy()
dff.rename(columns= {"UNIVERSE":"ticker",
                    'LSS_TICKER':'lionshares_ticker',
                    'FFD_BMK_ID':'bench_id'
                    },inplace=True)
dfd= dfd.set_index('ticker')
dff = dff.set_index('ticker')
dff

,DATE,lionshares_ticker,bench_id
ticker,,,
VTSMX,20230731,VTSMX-US,CRSPTM
VFINX,20230731,None,SP50
FXAIX,20230731,FXAIX-US,SP50
SPY,20230731,SPY-US,SP50
IVV,20230731,IVV-US,SP50
...,...,...,...
QDISX,20230731,None,MS664204
SGLC,20230731,SGLC-US,None
F000013LTI,20230731,None,None


In [3]:
dfd = dfd.join(dff[['lionshares_ticker','bench_id']],how='left')
dfd = dfd.dropna()

dfd = dfd[dfd['bench_id'].isin(['SP50','R.1000','R.1000G','R1000V','R.2000','R.2000G','R2000V','R.3000','R.3000G','R3000V','MS664204','CRSPTM','OEX','MS126220','892400','R.MID','R.MIDG','R.MIDV','98900023','CRSPMI','990100','984000','MS700727','899800'])]

In [4]:

dfd['pa_ticker'] = 'LION:'+dfd['lionshares_ticker']
dfd['pa_bench'] = 'BENCH:'+dfd['bench_id']

In [15]:
df_f = dfd.head(100)

pa_doc.run_calc(portfolios = df_f['pa_ticker'].to_list(),benchmarks= df_f['pa_bench'].to_list(),start_date = '0M-1AY',
                            end_date = '0M')

                            
df_full= pd.DataFrame()
for x in list(pa_doc.data.keys()):
    temp = pa_doc.data[x][0]
    temp['port_bench'] = x
    df_full = pd.concat([df_full,temp])

pattern = r'LION:(.*?)BENCH'
df_full['lionshares_ticker'] = df_full['port_bench'].str.extract(pattern, expand=False)
df_full.head()

,Symbol,total0,Beginning+Price,Ending+Price,Price+Change+(%),Ticker-Region,Port.+Beginning+Weight,Port.+Ending+Weight,Total+Return,Contribution+To Return,Port.+Ending+Market+Value,Bench.+Total+Return,Bench.+Total+Return+( Local ),Bench.+Contribution+To Return,Bench.+Contribution+to Return+( Local ),Bench.+Ending+Weight,Bench.+Beginning+Weight,port_bench,lionshares_ticker
0,,Total,,,13.548612,,100.0,100.0,13.981717,13.981717,1805173306.114,17.102878,17.102878,17.102878,17.102878,100.0,100.0,LION:ACFOX-USBENCH:R.1000G,ACFOX-US
1,67066G104,NVIDIA Corporation,181.630005,467.290009,157.275794,NVDA-US,3.112997,7.82844,157.483359,6.23363,141316907.22,157.420963,157.420963,4.207451,4.207451,5.288649,2.368783,LION:ACFOX-USBENCH:R.1000G,ACFOX-US
2,46120E602,"Intuitive Surgical, Inc.",230.169998,324.399994,40.939318,ISRG-US,3.477034,3.420285,40.939318,1.470025,61742078.8,40.939356,40.939356,0.209669,0.209669,0.53962,0.410782,LION:ACFOX-USBENCH:R.1000G,ACFOX-US
3,75886F107,"Regeneron Pharmaceuticals, Inc.",581.690002,741.909973,27.543855,REGN-US,3.070179,3.573813,27.543855,1.232681,64513525.96,27.543934,27.543934,0.019312,0.019312,0.024891,0.047667,LION:ACFOX-USBENCH:R.1000G,ACFOX-US
4,02079K107,Alphabet Inc. Class C,116.639999,133.110001,14.120382,GOOG-US,8.152466,8.677054,14.120382,1.162294,156635861.4,14.120411,14.120411,0.291224,0.291224,3.26513,3.018963,LION:ACFOX-USBENCH:R.1000G,ACFOX-US


In [13]:
df_full.to_csv('nividia_pa_contribution_vs_bench_dataset_20230828.csv')